# <strong> 1. Prerequisites</strong>

In [1]:
!pip install scipy==1.7
!pip install smt # for Latin Hypercube sampling
!pip install pandas

  Using cached scipy-1.7.0-cp39-cp39-macosx_10_9_x86_64.whl (32.1 MB)
  Using cached numpy-1.22.4-cp39-cp39-macosx_10_15_x86_64.whl (17.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.9.2 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
contourpy 1.3.0 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
You should consider upgrading via the '/Users/seangastinov/.pyenv/versions/3.9.10/bin/python3.9 -m pip install --upgrade pip' command.
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_x86_64.whl (6.9 MB)
  Usi

In [2]:
import os
import sys
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import time
import datetime
np.random.seed(42) # set a seed for the random generator
from smt.sampling_methods import LHS
# scipy
from scipy.integrate import quad_vec  # quad_vec allows to compute integrals accurately
from scipy.stats import norm
from scipy.stats import qmc # to perform Latin Hypercube Sampling (LHS) 



# <strong> 2. European Call & Put prices </strong>

## <Strong><font>2.1. Implement the characteristic function</font></Strong>

In [ ]:
def beta_function(u, tau, sigma, rho, kappa):
    return kappa - 1j * u * sigma * rho

def alpha_hat_function(u):
    return -0.5 * u * (u + 1j)

def d_function(u, tau, sigma, rho, kappa):
    gamma = 0.5 * sigma**2
    beta = beta_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)
    return np.sqrt(beta**2 - 4 * alpha_hat * gamma)

def g_function(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    return (beta - d) / (beta + d)

def A_function(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    common_term = np.exp(-d*tau)
    A_u = (kappa * theta / (sigma**2)) * ((beta-d)*tau - 2*np.log((g*common_term-1) / (g-1)))    
    return A_u

def B_function(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    common_term = np.exp(-d*tau)
    B_u = ((beta-d) / (sigma**2)) * ((1 - common_term) / (1 - g*common_term))
    return B_u


In [ ]:
def heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0):

    beta = beta_function(u, tau, sigma, rho, kappa)    
    #alpha_hat = alpha_hat_function(u)    
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    common_term = np.exp(-d*tau)
    A = A_function(u, tau, theta, sigma, rho, kappa)
    B = B_function(u, tau, sigma, rho, kappa)

    return np.exp(A + B * v0)

## <Strong><font>2.2. Perform numerical integration</font></Strong>: using scipy integrate quad_vec function

In [ ]:
def integral_price(m, tau, theta, sigma, rho, kappa, v0):
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m)*heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return integ

## <Strong><font>2.3. Calculate European Call price </font></Strong>

In [ ]:
def call_price(k, tau, S0, r, theta, sigma, rho, kappa, v0):
    m = np.log(S0/k) + r*tau #log-moneyness forward
    integ = integral_price(m, tau, theta, sigma, rho, kappa, v0)  
    price = S0 - k * np.exp(-r*tau) * integ  /np.pi
         
    return price

## <Strong><font>2.4. Calculate European put price </font></Strong>: using the call-put parity relation


In [ ]:
def put_price(k, tau, S0, r, theta, sigma, rho, kappa, v0):
    price = call_price(k, tau, S0, r, theta, sigma, rho, kappa, v0)- S0 + k * np.exp(-r*tau)
    return price

## 2.5. Calculate the Normalised Forward Put Price $\hat{P}$
$\hat{P}(t,S_t,v_t) = \frac{e^{rT}}{K} P(t,S_t,v_t)$

In [ ]:
def norm_forw_put_price(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r*tau #log-moneyness forward
    integ = integral_price(m, tau, theta, sigma, rho, kappa, v0)
    return 1 - (1/np.pi) * integ

# <strong> 3. Differential of the Normalised Forward Put Price w.r.t inputs </strong> 

In [ ]:
def integral_delta(m, tau, theta, sigma, rho, kappa, v0):
    integrand = (lambda u: 
        np.real((1j*u + 0.5) * np.exp((1j*u + 0.5)*m)*heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return integ

In [ ]:
def term_interm_theta(u, tau, sigma, rho, theta, kappa, v0):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)

    term1 = kappa*theta * (beta-d+2*((d*g*np.exp(-d*tau)) / ((g*np.exp(-d*tau))-1))) /(sigma**2)
    term2 = v0*(beta-d)*d * np.exp(-d*tau)*(1-g)/((sigma**2)*(1-g*np.exp(-d*tau))**2)
    term = term1 + term2

    return term    

In [ ]:
def integral_differential_phi_tau(m, tau, sigma, rho, theta, kappa, v0):

    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m)*heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0) * 
                term_interm_theta(u - 0.5j, tau, sigma, rho, theta, kappa, v0))  /(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return integ

## 3.1. $\frac{\partial \hat{P}}{\partial \theta}$

In [ ]:
def differential_wrt_theta(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r * tau     #log-moneyness forward
    if theta == 0.:
        theta = 0.00000001
    
    integrand = (lambda u: 
        np.real((1/theta) * np.exp((1j*u + 0.5)*m)* A_function(u - 0.5j, tau, theta, sigma, rho, kappa) * heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ

## 3.2. $\frac{\partial \hat{P}}{\partial lm}$
lm is the log-moneyness

In [ ]:
def differential_wrt_lm(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r * tau #log-moneyness forward
    res = - (1/np.pi) * integral_delta(m, tau, theta, sigma, rho, kappa, v0)
    return res

## 3.3. $\frac{\partial \hat{P}}{\partial v_o}$

In [ ]:
def differential_wrt_v0(lm, r, tau, theta, sigma, rho, kappa, v0):    
    m = lm + r * tau #log-moneyness forward
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m)* B_function(u - 0.5j, tau, sigma, rho, kappa) * heston_charact_funct(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ

## 3.4. $\frac{\partial \hat{P}}{\partial \tau}$

In [ ]:
def differential_wrt_tau(lm, r, tau, theta, sigma, rho, kappa, v0):    
    m = lm + r * tau #log-moneyness forward
    integ1 = integral_delta(m, tau, theta, sigma, rho, kappa, v0)
    integ2 = integral_differential_phi_tau(m, tau, sigma, rho, theta, kappa, v0)   

    return (-1/np.pi) * (r * integ1 + integ2)

## 3.5. $\frac{\partial \hat{P}}{\partial \kappa}$

In [ ]:
def differential_A_kappa(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    A_u = A_function(u, tau, theta, sigma, rho, kappa)

    res = A_u/kappa - (kappa * theta / (d * (sigma**2))) * (-d*tau+tau*beta+4*g/(g-1)+2*g*np.exp(-d*tau)*(2+tau*beta)/(1-g*np.exp(-d*tau)))
    return res

In [ ]:
def differential_B_kappa(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    B_u = B_function(u, tau, sigma, rho, kappa) 

    res = -B_u/d + (1/d) * (tau*beta*np.exp(-d*tau)*B_u/(1-np.exp(-d*tau)) - g*np.exp(-d*tau)*(2+tau*beta)*B_u/(1-g*np.exp(-d*tau)))
    return res

In [ ]:
def differential_phi_kappa(u, tau, theta, sigma, rho, kappa, v0):
    return heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0) * (differential_A_kappa(u, tau, theta, sigma, rho, kappa) +
                                                                         v0 * differential_B_kappa(u, tau, theta, sigma, rho, kappa))


In [ ]:
def differential_wrt_kappa(lm, r, tau, theta, sigma, rho, kappa, v0): 
    m = lm + r*tau   #log-moneyness forward   
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m) * differential_phi_kappa(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ   

## 3.6. $\frac{\partial \hat{P}}{\partial \rho}$

In [ ]:
def differential_A_rho(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)

    res = (kappa*theta*1j*u/(d*sigma)) * (tau*(beta-d)-2*g*(np.exp(-d*tau)*(2+tau*beta)/(g*np.exp(-d*tau)-1)-2/(g-1)))
    return res 

In [ ]:
def differential_B_rho(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    B_u = B_function(u, tau, sigma, rho, kappa) 

    res = (1j*u*sigma*B_u/d) * (1+np.exp(-d*tau)*(-tau*beta/(1-np.exp(-d*tau))+g*(2+tau*beta)/(1-g*np.exp(-d*tau))))
    return res

In [ ]:
def differential_phi_rho(u, tau, theta, sigma, rho, kappa, v0):
    return heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0) * (differential_A_rho(u, tau, theta, sigma, rho, kappa) +
                                                                         v0 * differential_B_rho(u, tau, sigma, rho, kappa))

In [ ]:
def differential_wrt_rho(lm, r, tau, theta, sigma, rho, kappa, v0):
    m = lm + r*tau  #log-moneyness forward
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m) * differential_phi_rho(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ   

## 3.7. $\frac{\partial \hat{P}}{\partial \sigma}$

In [ ]:
def differential_ln_sigma(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)    

    res= ((2j*u*rho*((beta**2)-(d**2))+4*beta*alpha_hat*sigma)*((g-1)*np.exp(-d*tau)-g*np.exp(-d*tau)+1)+
          (g-1)*np.exp(-d*tau)*tau*g*((beta+d)**2)*(1j*u*rho*beta+2*alpha_hat*sigma))/(d*((beta+d)**2)*(g-1)*(g*np.exp(-d*tau)-1))
    return res

In [ ]:
def differential_A_sigma(u, tau, theta, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    A_u = A_function(u, tau, theta, sigma, rho, kappa)
    diff_ln_sigma = differential_ln_sigma(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)

    res = -2*A_u/sigma + kappa*theta/(sigma**2)*(-1j*u*tau*rho+tau*(1j*u*rho*beta+2*alpha_hat*sigma)/d - 2*diff_ln_sigma)
    return res 

In [ ]:
def differential_quotient_sigma(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)

    res = ((np.exp(-d*tau)*(1j*u*rho*beta+2*alpha_hat*sigma))*(-tau*((beta+d)**2)*(1-g*np.exp(-d*tau))+(1-np.exp(-d*tau))*(2*beta+tau*g*((beta+d)**2)))
     - 2j*u*rho*(d**2)*np.exp(-d*tau)*(1-np.exp(-d*tau))) / (d*((beta+d)**2)*((1-g*np.exp(-d*tau))**2))
    return res

In [ ]:
def differential_B_sigma(u, tau, sigma, rho, kappa):
    beta = beta_function(u, tau, sigma, rho, kappa)
    d = d_function(u, tau, sigma, rho, kappa)
    g = g_function(u, tau, sigma, rho, kappa)
    alpha_hat = alpha_hat_function(u)
    
    diff_quot_sigma = differential_quotient_sigma(u, tau, sigma, rho, kappa)
    res = ((1-np.exp(-d*tau))*(sigma*(-1j*u*rho*d+1j*u*rho*beta+2*alpha_hat*sigma)-2*d*(beta-d))) / (d*(sigma**3)*(1-g*np.exp(-d*tau))) + (beta-d)/(sigma**2)*diff_quot_sigma
    return res

In [ ]:
def differential_phi_sigma(u, tau, theta, sigma, rho, kappa, v0):
    return heston_charact_funct(u, tau, theta, sigma, rho, kappa, v0) * (differential_A_sigma(u, tau, theta, sigma, rho, kappa) +
                                                                         v0 * differential_B_sigma(u, tau, sigma, rho, kappa))

In [ ]:
def differential_wrt_sigma(lm, r, tau, theta, sigma, rho, kappa, v0):  
    m = lm + r*tau  #log-moneyness forward      
    integrand = (lambda u: 
        np.real(np.exp((1j*u + 0.5)*m) * differential_phi_sigma(u - 0.5j, tau, theta, sigma, rho, kappa, v0))/(u**2 + 0.25))

    integ, err = quad_vec(integrand, 0, np.inf)
    return (-1/np.pi) * integ   

## 3.4. $\frac{\partial \hat{P}}{\partial r}$

In [ ]:
def differential_wrt_r(lm, r, tau, theta, sigma, rho, kappa, v0):
    diff_wrt_lm = differential_wrt_lm(lm, r, tau, theta, sigma, rho, kappa, v0)
    return tau * diff_wrt_lm

# <strong> 4. Data generation </strong>

## <Strong><font> 4.1. Inputs generation </font></Strong> 

Latin Hypercube Sampling

$\theta$, $\sigma$, $\kappa$, $\rho$ parameters, $v_0$, Log moneyness $lm=\ln(S_0/K)$, the time to maturity $\tau$ and $r$ will be sampled via Latin Hypercube Sampling (LHS) technique. 

In the following, we will use two modules to perform the LHS: *scipy.stats.qmc* and *smt.sampling_methods.LHS*. We will calculate the exact time every program takes to sample 10 points and keep the one with less time to generate the inputs.

In [ ]:
# LHS using smt.sampling_methods.LHS

# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)

start_time = time.clock()

bounds = np.array([[-2.,2.], [-0.01,0.10], [0.05,20], [0.0,1.], [0.1,2.], [-0.90,0.0], [0.005,3.], [0.,1.]])
sampling = LHS(xlimits=bounds, random_state=42)

n_samples = 10
samples = sampling(n_samples)

print("The exact time  the program takes to finish running: %s seconds " % (time.clock() - start_time))
#print(samples)

The exact time  the program takes to finish running: 0.0020979999999966026 seconds 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


In [ ]:
# LHS using scipy.stats.qmc
# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)

start_time = time.clock()  

sampler = qmc.LatinHypercube(d=8, seed=42) # 6 variables
samples = sampler.random(n=10) # n samples

lower_bounds = [-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.] 
upper_bounds = [2., 0.10, 20., 1.0, 2., 0.0, 3., 1.]
inputs_array = qmc.scale(samples, lower_bounds, upper_bounds)

print("The exact time  the program takes to finish running: %s seconds " % (time.clock() - start_time))
#print(inputs_array)

The exact time  the program takes to finish running: 0.0017220000000008895 seconds 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


Note that *scipy.stats.qmc* module takes less time than *smt.sampling_methods.LHS*, therefore, inputs generation will be performed using *scipy.stats.qmc* module. 

We will generate 100 000 (100K) samples. 

In [ ]:
def generate_inputs(d, n, lower_bounds, upper_bounds, feller=False):
    
    sampler = qmc.LatinHypercube(d, seed=42) # d variables    
    samples = sampler.random(n) # n samples
    inputs_array = qmc.scale(samples, lower_bounds, upper_bounds)
    
    if feller:
        #Selecting samples that satisfy the Feller condition        
        inputs_array = inputs_array[np.where(2*inputs_array[:,6]*inputs_array[:,3] > (inputs_array[:,4])**2)]

    return inputs_array

## <Strong><font> 4.2. Labels and their differentials w.r.t inputs generation </font></Strong> 

In [ ]:
def generate_labels_difflabels(inputs_array):
    start_time = time.clock()

    # array containing labels and its differentials
    labels_difflabels_array = np.zeros((inputs_array.shape[0],inputs_array.shape[1]+1))

    for row in range(inputs_array.shape[0]):

        lm = inputs_array[row][0] 
        r = inputs_array[row][1]
        tau = inputs_array[row][2]
        theta = inputs_array[row][3]
        sigma = inputs_array[row][4]
        rho = inputs_array[row][5]      
        kappa = inputs_array[row][6]
        v0 = inputs_array[row][7]    

        p_hat = norm_forw_put_price(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_lm = differential_wrt_lm(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_r = differential_wrt_r(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_tau = differential_wrt_tau(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_theta = differential_wrt_theta(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_sigma = differential_wrt_sigma(lm, r, tau, theta,sigma, rho, kappa, v0)
        diff_wrt_rho = differential_wrt_rho(lm, r, tau, theta,sigma, rho, kappa, v0)  
        diff_wrt_kappa = differential_wrt_kappa(lm, r, tau, theta,sigma, rho, kappa, v0)    
        diff_wrt_v0 = differential_wrt_v0(lm, r, tau, theta,sigma, rho, kappa, v0)             

        labels_difflabels_array[row][0] = p_hat 
        labels_difflabels_array[row][1] = diff_wrt_lm
        labels_difflabels_array[row][2] = diff_wrt_r
        labels_difflabels_array[row][3] = diff_wrt_tau
        labels_difflabels_array[row][4] = diff_wrt_theta
        labels_difflabels_array[row][5] = diff_wrt_sigma
        labels_difflabels_array[row][6] = diff_wrt_rho
        labels_difflabels_array[row][7] = diff_wrt_kappa
        labels_difflabels_array[row][8] = diff_wrt_v0          

    print("The exact time  the program takes to finish running:", time.strftime('%H:%M:%S', time.gmtime(time.clock() - start_time))) 
    return labels_difflabels_array


## <Strong><font> 4.3. Dataset generation </font></Strong> 

In [ ]:
def generate_data(d, n, lower_bounds, upper_bounds, feller, first_samples=100000):

    inputs_array = generate_inputs(d, n, lower_bounds, upper_bounds, feller)
    if feller:
        #extract first 100000 samples
        #inputs_array=inputs_array[0:100000, :]
        inputs_array=inputs_array[0:first_samples, :]

    labels_difflabels_array = generate_labels_difflabels(inputs_array)

     
    # Convert the inputs array to a dataframe
    df_inputs = pd.DataFrame(inputs_array, columns=['lm', 'r', 'tau', 'theta', 'sigma', 'rho', 'kappa', 'v0'])

    # Convert the array containing labels and its differentials to a dataframe
    df_labels_difflabels = pd.DataFrame(labels_difflabels_array, columns=["p_hat", "diff wrt lm", "diff wrt r", "diff wrt tau", "diff wrt theta",
                                                "diff wrt sigma", "diff wrt rho", "diff wrt kappa", "diff wrt v0"])
    # put the whole data into a dataframe
    df_dataset = pd.concat([df_inputs, df_labels_difflabels], axis=1)     
    
        
    return df_dataset

### 4.3.1 Data generation when the Feller condition is breached

The following algorithm takes 10 hours, 34 minutes and 24 seconds.

In [ ]:
# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)
#lower_bounds = [-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.] 
#upper_bounds = [2., 0.10, 20., 1.0, 2., 0.0, 3., 1.]

df_nofeller = generate_data(d=8, n=100000,                                      
                                     lower_bounds=[-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.], 
                                     upper_bounds=[2., 0.10, 20., 1.0, 2., 0.0, 3., 1.],
                                     feller=False)

# # Saving datasets into CSV files
os.makedirs('data', exist_ok=True)
df_nofeller.to_csv('data/dataset_100K_nofeller.csv', sep=',', index=False)

### 4.3.2 Data generation when the Feller condition is satisfied 

The following algorithm takes 9 hours, 14 minutes and 51 seconds.

In [ ]:
# A sample consists of a row in the form (lm, r, tau, theta, sigma, rho, kappa, v0)
#lower_bounds = [-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.] 
#upper_bounds = [2., 0.10, 20., 1.0, 2., 0.0, 3., 1.]

df_feller = generate_data(d=8, n=100000, 
                                   lower_bounds=[-2., -0.01, 0.05, 0.0, 0.1, -0.90, 0.005, 0.], 
                                   upper_bounds=[2., 0.10, 20., 1.0, 2., 0.0, 3., 1.],
                                   feller=True)

# # Saving datasets into CSV files
os.makedirs('data', exist_ok=True)
df_feller.to_csv('data/dataset_100K_feller.csv', sep=',', index=False)